In [ ]:
pip install jumanji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.5/583.5 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for jumanji: filename=jumanji-0.3.1-py3-none-any.whl size=750375 sha256=2d0cb7cf538d3965c9d14623932184ebb78eb6150442d21b14e440ab14b22981
  Stored in directory: /root/.cache/pip/wheels/31/29/10/884df072d319d2522cfff8b9c3d61fe60ab3c31e2a054d3e5d
Successfully built jumanji


In [ ]:
import jumanji
import jax.numpy as jnp
import jax
import random
import os
from collections import namedtuple, deque
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense
import numpy as np
import matplotlib.pyplot as plt
from time import sleep

In [ ]:
# Main framework taken from https://github.com/ultronify/dqn-from-scratch-with-tf2/blob/master/

class ReplayBuffer:

  def __init__(self):
    self.memory = deque(maxlen=10000)
    self.capacity = 10000

  def __len__(self):
    return len(self.memory)

  def store(self, state, next_state, reward, action, done):
    '''
    Records a single step of game play experience
    PARAM -
    state: current game state
    next_state: game state after taking action
    reard: reward taking action at the current state brings
    action: action taken at the current state
    done: boolean to indicate if game is finished after taking action
    RETURNS - N/A
    '''
    if len(self.memory) > self.capacity:
      del self.memory[0]
    self.memory.append((state, next_state, reward, action, done))


  def sample(self):
    '''
    Samples a batch of gameplay experiences for training
    PARAM - None
    RETURNS - list of gameplay experiences
    '''
    batch_size = min(128, len(self.memory))
    sample_batch = random.sample(self.memory, batch_size)
    state_batch = []
    next_state_batch = []
    reward_batch = []
    action_batch = []
    done_batch = []
    for experience in sample_batch:
      state_batch.append(experience[0])
      next_state_batch.append(experience[1])
      reward_batch.append(experience[2])
      action_batch.append(experience[3])
      done_batch.append(experience[4])

    return  np.array(state_batch), np.array(next_state_batch), np.array(reward_batch), np.array(action_batch), np.array(done_batch)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def unravel(state, action_index):
  array = np.array(state.action_mask)
  rotation, col_index = np.unravel_index(action_index, array.shape)
  action = [rotation, col_index]
  return action

class DqnAgent:
  '''
  Create DQN agent class
  '''

  def __init__(self):
    self.q_net = self.build_dqn_model()
    self.target_q_net = self.build_dqn_model()

  @staticmethod
  def build_dqn_model():
    '''
    Builds deep neural network to predict Q values for all possible actions given a state.
    Input should have shape of the state and the output should have the same shape as action space
    RETURNS - Q network
    '''
    q_net = Sequential()
    # Adds fully connected layer with 128 units and uses rectified linear unit activation function. he_uniform initliazes weight of layer
    q_net.add(Dense(256, input_dim = 82, activation='relu', kernel_initializer='he_uniform'))
    q_net.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    q_net.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    # 40 actions so 40 different outputs
    q_net.add(Dense(24, activation='linear', kernel_initializer='he_uniform'))

    # opt = keras.optimizers.Adam(learning_rate=0.001)
    q_net.compile(loss='mse', optimizer = 'adam')
    return q_net

  def convert_state(self,state):
    '''
    Convert state into observation variable that can be passed into neural network
    PARAM -
    state: current game state
    RETURNS - observation variable
    '''

    grid = state.grid_padded.flatten().tolist()
    tetromino = state.tetromino_index.flatten().tolist()
    # print(grid)
    # print(tetromino)
    obs_variable = np.asarray(grid+tetromino)
    # print("Current input dimensions:")
    # print(np.shape(obs_variable))
    # print(obs_variable)

    return obs_variable


  def policy(self,state, curr_epsiode_num):
    '''
    Takes state from environment and returns an action that has the highest q value using epsilon_greedy
    PARAM -
    state: current game state
    RETURNS - action
    '''
    # Matches state into array
    state_array = self.convert_state(state)
    # Convert to into tensorflow tensor
    state_input = tf.convert_to_tensor(state_array[None, :], dtype=tf.float32)
    # Grabs Q values for all possible actions in current state
    action_q = np.array(self.q_net(state_input))
    # Allows only legal moves
    action_mask = np.array(state.action_mask).flatten()
    # Checks if terminal state is reached when there are no more legal moves
    terminal = np.where(action_mask)[0]
    if len(terminal) == 0:
      return False, False
    # Calculate epsilon based on the current episode number
    epsilon = max(0.001, 1.0 - 0.01 * curr_epsiode_num)  # You can adjust the decay rate
    if np.random.rand() < epsilon: # This is the epsilon
      action, action_index = self.random_action(state)
      return action, action_index
    else:
      # Legal moves will have a value of 1 while illegal moves have a value of 0
      locater = action_mask.astype(int)
      # Ensures illegal moves remain illegal
      max_q = np.absolute(action_q).max()
      action_q[:,locater == 0] -= (max_q + 9999999999)
      # Grabs index of action and converts into suitable type
      action_index = np.argmax(action_q)
      action = unravel(state,action_index)
      return action, action_index

  def get_valid_actions(self,state):
    array = np.array(state.action_mask)
    # Flatten the array
    flattened_array = array.flatten()
    # Get the indices of true values, which are the number of valid actions in the current state
    true_indices = np.where(flattened_array)[0]

    return true_indices

  def random_action(self, state):
    array = np.array(state.action_mask)
    true_indices = self.get_valid_actions(state)
    if len(true_indices) == 0:
      return False
    random_index = np.random.choice(true_indices)
    # Convert the random index to two-dimensional coordinates
    rotation, col_index = np.unravel_index(random_index, array.shape)
    return [rotation,col_index], random_index

  def update_network(self):
    '''
    Updates current q network with q_net which brings all the training in q_net with target_q_net
    '''
    self.target_q_net.set_weights(self.q_net.get_weights())

  def train(self,batch):
    '''
    Trains underlying network with batch of gameplay experineces to help it predict Q values
    PARAM -
    Batch: batch of experiences
    RETURNS: Traning loss
    '''
    # Copying the batch over
    state_batch, next_state_batch, reward_batch, action_batch, done_batch = batch
    # Running states through the q_net gives output Q values for the states
    current_q = self.q_net(state_batch).numpy()
    # Copy over Q values for actions that weren't chosen
    target_q = np.copy(current_q)
    # Get the max Q values of states after transition by running next_state through target_q_net and take max Q values for all actions for each sample
    next_q = self.target_q_net(next_state_batch).numpy()
    max_next_q = np.amax(next_q, axis=1)
    # Update Q value of action taken with max Q value of next state plus intermediate reward from the action taken
    for i in range(state_batch.shape[0]):
      target_q_val = reward_batch[i].astype(float)
      action_index = action_batch[i]
      # print(action_index)
      if not done_batch[i]:
        target_q_val += 0.95 * max_next_q[i]
      target_q[i][action_index] = target_q_val
    # Train q_net with target Q values
    training_his = self.q_net.fit(x = state_batch, y=target_q)
    loss = training_his.history['loss']
    return loss

In [ ]:
def evaluate_training(env, agent):
  '''
  Evaluates performance of DQN agent and calculates average reward
  PARAM -
  env: game environment
  agent: DQN agent
  RETURNS: Average reward across episodes
  '''
  total_reward = 0.0
  episodes_to_play = 5
  for i in range(episodes_to_play):
    key = jax.random.PRNGKey(1)
    state, timestep = jax.jit(env.reset)(key)
    done = False
    episode_reward = 0.0
    while not done:
      action, action_index = agent.policy(state,999)
      if action is False:
        done = True
        break
      next_state, next_timestep = jax.jit(env.step)(state, action)
      episode_reward += next_state.reward
      state = next_state
    total_reward += episode_reward
  average_reward = total_reward / episodes_to_play
  return average_reward

def collect_experiences(env, agent, buffer, curr_episode):
  '''
  Collect gameplay experiences by playing with env and store experiences in buffer
  '''
  key = jax.random.PRNGKey(1)
  state, timestep = jax.jit(env.reset)(key)
  done = False
  terminal = False

  while not done:
    action, action_index = agent.policy(state, curr_episode)
    # Testing
    #action = agent.random_action(state)
    if action is False:
      done = True
      terminal = True
      break
    next_state, next_timestep = jax.jit(env.step)(state, action)
    buffer.store(agent.convert_state(state), agent.convert_state(next_state), next_state.reward, action_index, terminal)
    state = next_state

def train_model(max_episodes = 200):
  '''
  Trains DQN agent to play game
  RETURNS: None
  '''
  save_interval_1 = 50
  save_interval_2 = 5
  Results = []

  # Specify the directory to save the files
  save_dir = "saved_models_and_results"
  if not os.path.exists(save_dir):
    os.makedirs(save_dir)

  agent = DqnAgent()
  buffer = ReplayBuffer()
  # Instantiate tetris environment using registry
  env = jumanji.make('Tetris-v0', num_rows = 6, num_cols = 6, time_limit = 2000)
  # env = jumanji.make('Tetris-v0')


  for episode_cnt in range(max_episodes):
    print("Current episode: {0}".format(episode_cnt))
    collect_experiences(env, agent, buffer, episode_cnt)
    gameplay_batch = buffer.sample()
    loss = agent.train(gameplay_batch)
    print('So far the loss is {0}'.format(loss))
    avg_reward = evaluate_training(env, agent)
    Results.append(avg_reward)
    print('So far the performance is {0}'.format(avg_reward))
    # Update target q net every __ episodes (currently 5)
    if episode_cnt % 2 == 0:
      agent.update_network()
    if episode_cnt % save_interval_2 == 0:
      print("Saving results")
      results_filename = os.path.join(save_dir, f"results_at_episode_{episode_cnt}.csv")
      with open(results_filename, "w") as results_file:
        results_file.write("Episode,Reward\n")
        for episode, reward in enumerate(Results, start=1):
          results_file.write(f"{episode},{reward}\n")
    if episode_cnt % save_interval_1 == 0:
      print("Saving model")
      # Save best model as an HDF5 model file
      DQN_filename = os.path.join(save_dir, f"DQN_Agent_at_episode_{episode_cnt}.h5")
      agent.q_net.save(DQN_filename)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
train_model()
print('No problems')

Current episode: 0


1/1 [==============================] - 0s 380ms/step - loss: 0.5218
So far the loss is [0.5218287706375122]
So far the performance is 0.0
Saving results
Saving model
Current episode: 1
1/1 [==============================] - 0s 5ms/step - loss: 6.8240
So far the loss is [6.824009418487549]
So far the performance is 0.0
Current episode: 2
1/1 [==============================] - 0s 6ms/step - loss: 11.9097
So far the loss is [11.909675598144531]
So far the performance is 0.0
Current episode: 3
1/1 [==============================] - 0s 6ms/step - loss: 9.5733
So far the loss is [9.573344230651855]
So far the performance is 0.0
Current episode: 4
1/1 [==============================] - 0s 6ms/step - loss: 7.4710
So far the loss is [7.471009731292725]
So far the performance is 40.0
Current episode: 5
2/2 [==============================] - 0s 5ms/step - loss: 8.3003
So far the loss is [8.300277709960938]
So far the performance is 40.0
Saving results
Current episode: 6
2/2 [=====================

KeyboardInterrupt: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')